Define the schema for the database

In [1]:
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string', 'required': True}
            }
        }
    }
}

Create "sample.osm" from "cosprings.osm" in order to have a smaller file to test the python code on

In [2]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM = "cosprings.osm" 

# Remove quotations below to create SAMPLE_FILE

SAMPLE_FILE = "sample.osm"

k = 30 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag and place in sample.osm

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')


'\nSAMPLE_FILE = "sample.osm"\n\nk = 30 # Parameter: take every k-th top level element\n\ndef get_element(osm_file, tags=(\'node\', \'way\', \'relation\')):\n    """Yield element if it is the right type of tag\n\n    Reference:\n    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python\n    """\n    context = iter(ET.iterparse(osm_file, events=(\'start\', \'end\')))\n    _, root = next(context)\n    for event, elem in context:\n        if event == \'end\' and elem.tag in tags:\n            yield elem\n            root.clear()\n\n\nwith open(SAMPLE_FILE, \'wb\') as output:\n    output.write(\'<?xml version="1.0" encoding="UTF-8"?>\n\')\n    output.write(\'<osm>\n  \')\n\n    # Write every kth top level element\n    for i, element in enumerate(get_element(OSM)):\n        if i % k == 0:\n            output.write(ET.tostring(element, encoding=\'utf-8\'))\n\n    output.write(\'</osm>\')\n'

Return the types of tags and the number of each

In [3]:
def count_tags(filename):
    # count how many of each tag type exist in the data
    tags = {}
    for event, elem in ET.iterparse(filename,events=('start',)):
        if elem.tag in tags.keys():
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags


Find the number of keys with the specified patterns: lower, lower_colon, problemchars, and other

In [4]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

keys = {}
def key_type(element, keys):
    # determine the types of keys that exist in the data
    if element.tag == "tag":
        k_val = element.get('k')
        if bool(lower.search(k_val)):
            keys['lower'] += 1
        elif bool(lower_colon.search(k_val)):
            keys['lower_colon'] += 1
        elif bool(problemchars.search(k_val)):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1    
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys



Find the number of users who have contributed to the OSM file

In [5]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if bool(element.get('uid')):
            users.add(element.attrib['uid'])
    return len(users)



Return the different values for key='amenity' and the number of each value

In [6]:
def amenities(osmfile):
    osm_file = open(osmfile, "r")
    amenity = {}
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "amenity":
                    if tag.attrib['v'] in amenity.keys():
                        amenity[tag.attrib['v']] += 1
                    else:
                        amenity[tag.attrib['v']] = 1
    osm_file.close()
    return amenity



Audit the street types to determine which types need to be programatically corrected. Add the types to be corrected programatically to mapping 

In [14]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Circle","Highway","Parkway","Terrace","Trafficway","Way","Loop","Alley",
            "Point"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_streets(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

audit_streets(OSM)





Define a procedure to programmatically replace mapping keys with the values.  For instance '1106 W 47th street' should return '1106 W 47th Street'.  

In [24]:
mapping = { "St": "Street","St.": "Street","Ct": "Court","Rd": "Road","Rd.": "Road","Ave": "Avenue","Ave.": "Avenue",
            "Blvd":"Boulevard","Blvd.":"Boulevard","Ct":"Court","Dr":"Drive","Dr.":"Drive","HWY":"Highway",
            "Hwy":"Highway","Ln":"Lane","Pkwy":"Parkway","RD":"Road","ST":"Street","STREET":"Street","Ter":"Terrace",
            "Trfy":"Trafficway","ave":"Avenue","circle":"Circle","ct":"Court","dr":"Drive","Pl":"Place","rd":"Road",
            "st":"Street","st.":"Street","street":"Street","ter":"Terrace","terrace":"Terrace"
            }

def update_street(name, mapping):
    s = street_type_re.search(name)
    if s:
        street_type = s.group()
        if (street_type not in expected) and (street_type in mapping.keys()):                                
            name = re.sub(street_type_re,mapping[street_type],name)                                
    return name



'1106 W 47th Street'

Audit postcodes to determine if any postcodes do not conform to the 5 digit format

In [25]:
def audit_postcodes(osmfile):
    osm_file = open(osmfile, "r")
    postcodes = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:postcode':
                    postcode = tag.attrib['v']
                    if len(postcode) != 5:
                        postcodes.add(postcode)
    osm_file.close()
    return postcodes



Define a procedure to correct postcodes programmatically that do not conform to the 5 digit standard.  For instance, 'CO 55555-4444' should be converted to '55555'.  Postcodes all conforming to a standardized format allows postcode queries to be more meaningful.

In [26]:
postcode_re = re.compile(r'ddddd', re.IGNORECASE)

def update_code(code):
    c = postcode_re.search(code)
    if c:
        codes = c.group()
        code = codes
    return code

update_code('KS 66210-1415')

'KS 66210-1415'

Audit phone numbers to catch any not conforming to the standard format '+1-555-555-5555'.

In [27]:
def audit_phone(osmfile):
    osm_file = open(osmfile, "r")
    phones = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'phone':
                    number = tag.attrib['v']
                    if len(number) != 15:
                        phones.add(number)
    osm_file.close()
    return phones


Define a procedure that will update phone numbers programmatically to the format defined above.  For instance, '(555) 555-5555' returns '+1-555-555-5555'.  This allows queries on phone numbers to be more valid.  

In [54]:
def is_phone(elem):
    # returns a Boolean value if the element key = "phone"
    return (elem.attrib['k'] == "phone")

def update_phone(phone):
    #Format all US phone numbers containing an area code to a standard format (+1-913-555-5555) 
    phone =  re.sub('[^0-9]', '', phone)
    if len(phone) <= 8:  # Add this caveat to ensure we don't try our formatting tool on phone numbers lacking area code
        return phone
    elif phone[0] != '1':
        return "+{}-{}-{}-{}".format(
        '1', phone[0:3], phone[3:6], phone[6:10])
    elif phone[0]=='1':
        return "+{}-{}-{}-{}".format(
        phone[0], phone[1:4], phone[4:7], phone[7:11])
        





Using the schema defined at the top of the code, write csv files to create the database.

In [29]:
import csv
import codecs
import cerberus

OSM_PATH = OSM

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for node in NODE_FIELDS:
            node_attribs[node] = element.attrib[node]

        for child in element:
            tag = {}
            if PROBLEMCHARS.search(child.attrib["k"]):
                continue
            elif LOWER_COLON.search(child.attrib["k"]):
                tag_type = child.attrib["k"].split(':',1)[0]
                tag_key = child.attrib["k"].split(':',1)[1]
                tag["key"] = tag_key
                if tag_type:
                    tag["type"] = tag_type
                else:
                    tag["type"] = 'regular'
                
                tag["id"] = element.attrib["id"]
                tag["value"] = child.attrib["v"]
                
            else:
                tag["value"] = child.attrib["v"]
                tag["key"] = child.attrib["k"]
                tag["type"] = "regular"
                tag["id"] = element.attrib["id"]
            if tag['key'] == "street":
                tag['value'] = update_street(tag['value'],mapping)
            if tag['key'] == "phone":
                tag['value'] = update_phone(tag['value'])
            if tag:
                    tags.append(tag)       
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for way in WAY_FIELDS:
            way_attribs[way] = element.attrib[way]
        for child in element:
            nd = {}
            tag = {}
            if child.tag == 'tag':
                if PROBLEMCHARS.search(child.attrib["k"]):
                    continue
                elif LOWER_COLON.search(child.attrib["k"]):
                    tag_type = child.attrib["k"].split(':',1)[0]
                    tag_key = child.attrib["k"].split(':',1)[1]
                    tag["key"] = tag_key
                    if tag_type:
                        tag["type"] = tag_type
                    else:
                        tag["type"] = 'regular'
                    tag["id"] = element.attrib["id"]
                    tag["value"] = child.attrib["v"]
                    
                else:
                    tag["value"] = child.attrib["v"]
                    tag["key"] = child.attrib["k"]
                    tag["type"] = "regular"
                    tag["id"] = element.attrib["id"]
                if tag['key'] == "street":
                    tag['value'] = update_street(tag['value'],mapping)
                if tag['key'] == "phone":
                    tag['value'] = update_phone(tag['value'])
                if tag:
                    tags.append(tag)
            elif child.tag == 'nd':
                nd['id'] = element.attrib["id"]
                nd['node_id'] = child.attrib["ref"]
                nd['position'] = len(way_nodes)
                
                if nd:
                    way_nodes.append(nd)
            else:
                continue        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)



def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                    
process_map(OSM,validate=False)